# RKI Corona Daten

Link: https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0/data

Herunterladen -> Tabelle  (Datei 173MB groß (Stand 01.02.2021))

Link zum Datensatz: https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv

## Gesamtdaten einlesen

In [1]:
import pandas as pd

link="https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv"
RKI_komplett = pd.read_csv(link)

RKI_komplett.head(5)

,ObjectId,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
0,1,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020/09/30 00:00:00+00,1001,"11.02.2021, 00:00 Uhr",0,-9,2020/09/30 00:00:00+00,0,1,0,Nicht übermittelt
1,2,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020/10/29 00:00:00+00,1001,"11.02.2021, 00:00 Uhr",0,-9,2020/10/29 00:00:00+00,0,1,0,Nicht übermittelt
2,3,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020/11/03 00:00:00+00,1001,"11.02.2021, 00:00 Uhr",0,-9,2020/11/03 00:00:00+00,0,1,0,Nicht übermittelt
3,4,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020/11/20 00:00:00+00,1001,"11.02.2021, 00:00 Uhr",0,-9,2020/11/19 00:00:00+00,0,1,1,Nicht übermittelt
4,5,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020/11/23 00:00:00+00,1001,"11.02.2021, 00:00 Uhr",0,-9,2020/11/18 00:00:00+00,0,1,1,Nicht übermittelt


## Deutschland rausfiltern

In [2]:
RKI_Deutschland = RKI_komplett.groupby(['Meldedatum']).agg({'AnzahlFall': ['sum']}).reset_index()
RKI_Deutschland['Meldedatum'] = pd.to_datetime(RKI_Deutschland['Meldedatum'],format='%Y/%m/%d')
RKI_Deutschland.columns = ['Meldedatum','Neue Fallzahlen']
RKI_Deutschland["Neue Fallzahlen"] = RKI_Deutschland["Neue Fallzahlen"].astype(int)
RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"] = RKI_Deutschland["Neue Fallzahlen"].rolling(7).mean().fillna(0)
RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"] = RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"].astype(int)
RKI_Deutschland["Faelle gesamt"] = RKI_Deutschland["Neue Fallzahlen"].cumsum(axis = 0) 
RKI_Deutschland.to_csv("RKI_Corona_Deutschland.csv", index=False)

RKI_Deutschland.head()

,Meldedatum,Neue Fallzahlen,Neue Fallzahlen Mittelwert (7 Tage),Faelle gesamt
0,2020-01-02 00:00:00+00:00,6,0,6
1,2020-01-03 00:00:00+00:00,6,0,12
2,2020-01-04 00:00:00+00:00,3,0,15
3,2020-01-05 00:00:00+00:00,1,0,16
4,2020-01-06 00:00:00+00:00,2,0,18


## Bundesland rausfiltern

In [6]:
RKI_Bundesland = RKI_komplett.groupby(['Meldedatum','Bundesland']).agg({'AnzahlFall': ['sum']}).reset_index()
RKI_Bundesland.columns=["Meldedatum","Bundesland","Neue Fallzahlen"]
RKI_Bundesland['Meldedatum'] = pd.to_datetime(RKI_Bundesland['Meldedatum'],format='%Y/%m/%d')
RKI_Bundesland.to_csv("RKI_Corona_Bundeslaender.csv", index=False)
RKI_Bundesland

,Meldedatum,Bundesland,Neue Fallzahlen
0,2020-01-02 00:00:00+00:00,Brandenburg,3
1,2020-01-02 00:00:00+00:00,Nordrhein-Westfalen,3
2,2020-01-03 00:00:00+00:00,Brandenburg,1
3,2020-01-03 00:00:00+00:00,Nordrhein-Westfalen,5
4,2020-01-04 00:00:00+00:00,Nordrhein-Westfalen,3
...,...,...,...
5451,2021-02-10 00:00:00+00:00,Saarland,56
5452,2021-02-10 00:00:00+00:00,Sachsen,263
5453,2021-02-10 00:00:00+00:00,Sachsen-Anhalt,172
5454,2021-02-10 00:00:00+00:00,Schleswig-Holstein,328


## Leipzig rausfiltern

In [5]:
Leipzig = RKI_komplett.loc[RKI_komplett["Landkreis"] == "SK Leipzig"]
Leipzig = Leipzig.groupby(['Meldedatum']).agg({'AnzahlFall': ['sum']}).reset_index()
Leipzig['Meldedatum'] = pd.to_datetime(Leipzig['Meldedatum'],format='%Y/%m/%d')
Leipzig.columns = ['Meldedatum','Neue Fallzahlen']
Leipzig["Neue Fallzahlen Mittelwert (7 Tage)"] = Leipzig["Neue Fallzahlen"].rolling(7).mean().fillna(0).astype(int)
Leipzig["Neue Fallzahlen Summe (7 Tage)"] = Leipzig["Neue Fallzahlen"].rolling(7).sum().fillna(0).astype(int)
Leipzig["Faelle gesamt"] = Leipzig["Neue Fallzahlen"].cumsum(axis = 0) 
Einwohner = 605407  # Quelle: https://www.leipzig.de/news/news/leipzig-hat-jetzt-605407-einwohner/
Leipzig["7 Tage Indizdenz"] = Leipzig["Neue Fallzahlen Summe (7 Tage)"]/Einwohner*100000
Leipzig["7 Tage Indizdenz"] = Leipzig["7 Tage Indizdenz"].round(0)
Leipzig.to_csv("RKI_Corona_Leipzig.csv", index=False)
Leipzig

,Meldedatum,Neue Fallzahlen,Neue Fallzahlen Mittelwert (7 Tage),Neue Fallzahlen Summe (7 Tage),Faelle gesamt,7 Tage Indizdenz
0,2020-03-06 00:00:00+00:00,1,0,0,1,0.0
1,2020-03-08 00:00:00+00:00,1,0,0,2,0.0
2,2020-03-10 00:00:00+00:00,2,0,0,4,0.0
3,2020-03-11 00:00:00+00:00,2,0,0,6,0.0
4,2020-03-12 00:00:00+00:00,10,0,0,16,0.0
...,...,...,...,...,...,...
262,2021-02-06 00:00:00+00:00,35,61,430,14285,71.0
263,2021-02-07 00:00:00+00:00,6,58,408,14291,67.0
264,2021-02-08 00:00:00+00:00,17,55,388,14308,64.0
265,2021-02-09 00:00:00+00:00,69,52,366,14377,60.0
